In [35]:
import numpy as np
import matplotlib.pyplot as plt

In [36]:
keys = ['train', 'dev', 'test']

In [37]:
p, q, a = {}, {}, {}
for k in keys :
    file = open('CNN/cnn/' + k + '.txt').read().strip().split('\n\n')
    file = [x.split('\n') for x in file]
    p[k] = [x[2] for x in file]
    q[k] = [x[0] for x in file]
    a[k] = [x[1] for x in file]

In [48]:
from vectorizer import Vectorizer

In [49]:
vec = Vectorizer(min_df=7)

In [50]:
vec.fit(p['train'] + q['train'])

In [51]:
entities = {}
for k in p :
    entities[k] = []
    for x in p[k] :
        entities[k] += [y for y in x.split() if y.startswith('@entity')]
    
    entities[k] = set(entities[k])

In [52]:
for e in entities['train'] :
    vec.add_word(e)

In [59]:
vec.entity2idx = {}
i = 0
for w in vec.word2idx :
    if w.startswith('@entity') :
        vec.entity2idx[w] = i
        i += 1
vec.idx2entity = {v:k for k, v in vec.entity2idx.items()}

In [62]:
def generate_label_and_filter(p, q, a) :
    entities = np.zeros((len(vec.entity2idx),))
    for w in p.split() :
        if w.startswith('@entity') :
            entities[vec.entity2idx[w]] = 1
    
    return entities, vec.entity2idx[a]

In [66]:
vec.paragraphs = {}
vec.questions = {}
vec.entity_masks = {}
vec.answers = {}

for k in keys :
    vec.paragraphs[k] = vec.texts_to_sequences(p[k])
    vec.questions[k] = vec.texts_to_sequences(q[k])
    vec.entity_masks[k] = []
    vec.answers[k] = []
    for i in range(len(p[k])) :
        mask, answer = generate_label_and_filter(p[k][i], q[k][i], a[k][i])
        vec.entity_masks[k].append(mask)
        vec.answers[k].append(answer)

In [72]:
from torchtext.vocab import Vectors, GloVe, CharNGram, FastText
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
vectors = Vectors('wiki.simple.vec', url=url)

In [73]:
vec.word_dim = vectors.dim
import numpy as np
vec.embeddings = np.zeros((len(vec.idx2word), vec.word_dim))
for i, word in vec.idx2word.items() :
    vec.embeddings[i] = vectors[word].numpy()

In [74]:
import pickle
pickle.dump(vec, open('CNN/vec_cnn.p', 'wb'))